# Biopython (SARS-CoV-2) Notebook tutorial

This basic tutorial shows you how to identify an "unknown sequence" of DNA/RNA (which is derived from a coronavirus genome). The tutorial uses Biopython to identify. So, let's start to characterize the sequence.

### 1. Setup, Installation and Import

In [1]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass

     |████████████████████████████████| 2.3 MB 2.0 MB/s 


In [2]:
import os
import sys

from urllib.request import urlretrieve

import Bio
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

print("Python version:", sys.version_info)
print("Biopython version:", Bio.__version__)

Python version: sys.version_info(major=3, minor=7, micro=12, releaselevel='final', serial=0)
Biopython version: 1.79


Input file

In [3]:
input_file = "unknown-sequence.fasta"

fasta_my = ("https://raw.githubusercontent.com/katerinaoleynikova/Biopython/main/unknown-sequence.fasta")

if not os.path.exists(input_file):
    urlretrieve(fasta_my, input_file)

### 2. Basic genome properties

In [4]:
for record in SeqIO.parse(input_file, "fasta"):
    print(record.id)

Unknown_sequence


There is just one sequence with header "Unknown_sequence". # We are not dealing with many chromosomes, scaffolds or contigs.

Read the sequence

In [5]:
record = SeqIO.read(input_file, "fasta")
record.seq

Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA')

In [6]:
print("Sequence length (bp):", len(record))

Sequence length (bp): 29903


The seq length is ~30Kb. If this sequence represents an individual organism then it is very small / far too small for a typical eukaryote and too short for many microbes too (e.g. bacterial genomes are typically Mb). This indicates that this genome could be from a virus.

In [7]:
print("GC content (%) =", GC(record.seq))

GC content (%) = 37.97277865097148


The GC content is 0.38. So, the sequence is somewhat AT-rich, but within a normal range.

### 3. Comparison with other genome sequences

Let's use BLAST to align the unknown sequence to other annotated seqs in the NCBI nt database, which contains sequences from many different species from across the tree of life.

This may take ~ten minutes since we are doing an online search against many sequences (for larger queries, it would sensible to run BLAST locally instead; see Bio.Blast.Applications).

BLAST - program for finding homologues.

In [12]:
# The way 1: with a .fasta file
with open("unknown-sequence.fasta") as fasta_file :
  records = Bio.SeqIO.parse(fasta_file, "fasta")
  record = next(records)
  print(record.seq)

...


In [15]:
# The way 2: a short sequence #
%%time
result_handle = NCBIWWW.qblast("blastn", "nt", "ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTAT")

CPU times: user 295 ms, sys: 37.9 ms, total: 333 ms
Wall time: 1min 3s


In [27]:
# that used above
len("ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTAT")

469

Let's process the results with one of Biopython's generic parser.


In [16]:
blast_qresult = SearchIO.read(result_handle, "blast-xml")

In [17]:
print(blast_qresult)

Program: blastn (2.12.0+)
  Query: No (469)
         definition line
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|2118293194|gb|OK546350.1|  Severe acute respiratory ...
            1      1  gi|2105871194|gb|OK372407.1|  Severe acute respiratory ...
            2      1  gi|2105870908|gb|OK372385.1|  Severe acute respiratory ...
            3      1  gi|2105870672|gb|OK372367.1|  Severe acute respiratory ...
            4      1  gi|2098951190|gb|OK247119.1|  Severe acute respiratory ...
            5      1  gi|2098951128|gb|OK247114.1|  Severe acute respiratory ...
            6      1  gi|2098951004|gb|OK247104.1|  Severe acute respiratory ...
            7      1  gi|2098950905|gb|OK247096.1|  Severe acute respiratory ...
            8      1  gi|2098949943|gb|OK247021.1|  Severe acute respi

These descriptions are truncated, let's view them in full, for just the first 5 records.

In [18]:
[hit.description for hit in blast_qresult[:5]]

['Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/IA-Noblis-S762B33/2021 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), and ORF7b (ORF7b) genes, complete cds; ORF8 gene, complete sequence; and nucleocapsid phosphoprotein (N) and ORF10 protein (ORF10) genes, complete cds',
 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/TN-VUMC-000018/2020, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/TN-VUMC-000046/2020, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/TN-VUMC-000071/2020, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/VSP5512/2021 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (OR

That looks fairly conclusive without doing any quantitative analyses; it's already very likely we have a coronavirus genome here, specifically SARS2-CoV-2 that was the cause of the COVID19 pandemic.

Let's have a look at the first result in a bit more detail to check some of the alignment metrics.

In [19]:
first_hit = blast_qresult[0]

In [20]:
first_hit.description

'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/IA-Noblis-S762B33/2021 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), and ORF7b (ORF7b) genes, complete cds; ORF8 gene, complete sequence; and nucleocapsid phosphoprotein (N) and ORF10 protein (ORF10) genes, complete cds'

In [22]:
first_hsp = first_hit[0] # hsp - high scoring pairs
print(first_hsp.evalue, first_hsp.bitscore) # evalue - E value of highest scoring Blast return.

0.0 847.066


In [28]:
# E-value (Expect) - this value shows how random the resulting alignment is. This is the number of finds with the same or greater weight when searching the database for random sequences.
# The smaller the E-value, the better the match.
# A bit score is another prominent statistical indicator used in addition to the E-value in a BLAST output. The bit score measures sequence similarity independent of query sequence length and database size and is normalized based on the raw pairwise alignment score.
# The higher the bit-score, the better the sequence similarity

In [67]:
print(first_hsp.aln)

Alignment with 2 rows and 469 columns
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTC...TAT No
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTC...TAT gi|2118293194|gb|OK546350.1|


The alignment appears to have high quality and is not merely a spurious hit.

We could view/save the full sequence alignment, for illustration purposes. Here is just the first 100 characters in FASTA format.

In [70]:
print(format(first_hsp.aln, "fasta")[:100]) #

>No definition line
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCT
GTTCTCTAAACGAACTTTA


In [69]:
print(format(first_hsp.aln, "fasta"))

>No definition line
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCT
GTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACT
CACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATC
TTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTT
CGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAAC
ACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGG
AGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGG
CTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTAT
>gi|2118293194|gb|OK546350.1| Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/IA-Noblis-S762B33/2021 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), and ORF7b (ORF7b) genes, complete cds; ORF8 gene, complete sequence; and nucleocapsid phosphoprotein (N) and ORF10 protein (ORF10) genes, complete cds
ATTAAAGGTTTATACCTTCCCAG

### 4. Extraction of annotations on the matching genome sequence

Extract more structured metadata on the top matching sequence (the most homologous sequence using) NCBI Entrez via Biopython to extract GenBank file.

In [71]:
NCBI_id = first_hit.id.split('|')[3]
NCBI_id

'OK546350.1'

In [79]:
Entrez.email = "katen217@gmail.com"  # always tell NCBI you are

In [80]:
handle = Entrez.efetch(db="nucleotide", id= NCBI_id, retmode="text", rettype="gb") # file format .gb

In [81]:
genbank_record = SeqIO.read(handle, "genbank")
genbank_record

SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA'), id='OK546350.1', name='OK546350', description='Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/IA-Noblis-S762B33/2021 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), and ORF7b (ORF7b) genes, complete cds; ORF8 gene, complete sequence; and nucleocapsid phosphoprotein (N) and ORF10 protein (ORF10) genes, complete cds', dbxrefs=['BioProject:PRJNA718231', 'BioSample:SAMN22244798'])

p.s. There's a lot of information in the genbank record..

In [83]:
print("Is it single or double stranded and DNA or RNA virus?\n", genbank_record.annotations["molecule_type"])

Is it single or double stranded and DNA or RNA virus?
 RNA


In [84]:
print("What is the full NCBI taxonomy of this virus?\n", genbank_record.annotations["taxonomy"])

What is the full NCBI taxonomy of this virus?
 ['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus']


In [85]:
print("What are the relevant references/labs who generated the data?\n")

for reference in genbank_record.annotations["references"]:
    print(reference)

What are the relevant references/labs who generated the data?

location: [0:29890]
authors: Brinkac,L.M., Diepold,S., Mitchell,S., Sarnese,S., Kolakowski,L.F., Nelson,W.M. and Jennings,K.
title: Direct Submission
journal: Submitted (18-OCT-2021) Bioinformatics and Life Sciences, Noblis, 2002 Edmund Halley Drive, Reston, VA 20191, USA
medline id: 
pubmed id: 
comment: 



Now we can read up more about the virus and the source data through following these references and appropriate google searches.

Note that from the id, we could also find the record [here](https://www.ncbi.nlm.nih.gov/nuccore/OK546350.1/) on the NCBI website.

### 5. Protein level analysis

We might be interested in the gene/protein sequences, not just the entire genome. It is possible to retrieve the protein coding sequences (CDSs) from the Genbank record.

In [86]:
len(genbank_record.features)

55

In [87]:
{feature.type for feature in genbank_record.features}

{'CDS', 'gene', 'mat_peptide', 'misc_feature', 'source', 'stem_loop'}

In [89]:
CDSs = [feature for feature in genbank_record.features if feature.type == "CDS"]
len(CDSs)

11

Look at the first protein and extract the underlying sequence.

In [90]:
CDSs[0].qualifiers["gene"]

['ORF1ab']

In [91]:
protein_seq = Seq(CDSs[0].qualifiers["translation"][0])
protein_seq

Seq('MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLV...VNN')

In [92]:
print("Does the sequence begin with a start codon?\n", protein_seq.startswith("M"))

Does the sequence begin with a start codon?
 True


We can check roughly how this protein sequence relates to the underlying nucleotide sequence by looking at the DNA codon table.

In [93]:
print(CodonTable.unambiguous_dna_by_id[1])

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

However, we can't perform an exact "reverse translation" of course, since several amino acids are produced by the same codon. 

Note that if instead we started with the nucleotide sequence, then we could use Biopython's *.transcribe()* and *.translate()* functions to convert sequences from DNA to RNA and DNA to protein respectively.

In [95]:
print("Protein sequence length in amino acids:", len(protein_seq))

Protein sequence length in amino acids: 7096


It's a long protein for a virus. Let's check the annotation.

In [96]:
CDSs[0].qualifiers["product"]

['ORF1ab polyprotein']

So it looks like this is a polyprotein, which explains why it is a relatively long protein. Polyproteins are a typical feature of some viral genomes where smaller proteins are joined together providing a particular organization of the viral proteome.

### What's next?

Logical next steps at the genome level might include building a multiple sequence alignment from many cornavirus genomes (checkout the Biopython wrapers/parsers for `Clustal` and `Mafft` and `Bio.Align`/`Bio.parirwise2` plus `Bio.AlignIO`), building a phylogeny with an external tool like [iq-tree](http://www.iqtree.org/) and then viewing the tree with `Bio.Phylo`, the [ete3 toolkit](http://etetoolkit.org/), or [Jalview](https://www.jalview.org/).

Other protein level analyses could involve including building protein trees, annotating the proteins and visualization (e.g. `Bio.Graphics`), doing evolutionary rate analyses (e.g. `Bio.Phylo.PAML `), looking at protein structure, clustering and much more.

These kind of analyses can provide useful biological and epidemiological information that is very important for the coronavirus in an outbreak situation. For example, allowing tracking of how the outbreak spreads and indicating appropriate infection control measures, although caution in the interpretation of results is always required. See [Nextstrain](https://nextstrain.org/ncov) for an excellent resource of this kind.

Note, there is tons of other functionality in Biopython, this is just a very small fraction of the modules, see [Peter Cock's Biopython workshop](https://github.com/peterjc/biopython_workshop) and the extensive [official tutorial documentation](http://biopython.org/DIST/docs/tutorial/Tutorial.html).

p.s. This Notebook was made based on the Chris Rands's repository (https://github.com/chris-rands).